# Exercise 1. Socrata-Py
## Create a Dataset
Let's use [Socrata-py](https://github.com/socrata/socrata-py) to create a dataset from a CSV.

### Import Libraries

In [2]:
import os
import pandas as pd

from socrata.authorization import Authorization
from socrata import Socrata

## Setup Authentication
### Enter Education Credentials

Also, it is pow possible to set up API Keys and pass key id and secret values as Socrata user name and password or [api keys](https://socrataapikeys.docs.apiary.io) respectively

In [18]:
domain = 'macondointernal.demo.socrata.com'

user_name = ''
password = ''

# how you would pass credentials stored in environment variables
# user_name = os.environ['SOCRATA_KEY_ID']
# password = os.environ['SOCRATA_KEY_SECRET']

auth = Authorization(
  domain,
  user_name,
  password
)

socrata = Socrata(auth)

## Data
- Household Median Income data for all Arizona places has been saved to `/data/census_detail_household_median_income5.csv`
- Review the first and last 5 records
- Note that the Census returns certain negative values when a valid estimate is not available for given variable and geography. More on that, https://census.gov/data/developers/data-sets/acs-1year/notes-on-acs-estimate-and-annotation-values.html

In [19]:
dataset_name = 'Arizona Places Median Household Income'
dataset_description = 'source = file'
file_name = '../data/census_detail_household_median_income5.csv'
data = pd.read_csv(file_name)

# View first 5 records
data.head()

,name,type,variable_description,variable,value,annotation,year,date,geography_id,change_rate
0,"Aguila CDP, Arizona",place,Median Household Income,B19013_001E,33125.0,NaN,2011,2011-12-31,1600000US0400730,NaN
1,"Ajo CDP, Arizona",place,Median Household Income,B19013_001E,25181.0,NaN,2011,2011-12-31,1600000US0400870,NaN
2,"Ak Chin CDP, Arizona",place,Median Household Income,B19013_001E,-666666666.0,Either no sample observations or too few sampl...,2011,2011-12-31,1600000US0400940,NaN
3,"Ak-Chin Village CDP, Arizona",place,Median Household Income,B19013_001E,33083.0,NaN,2011,2011-12-31,1600000US0401090,NaN
4,"Alamo Lake CDP, Arizona",place,Median Household Income,B19013_001E,35938.0,NaN,2011,2011-12-31,1600000US0401170,NaN


In [20]:
# View last 5 records to check that we have most recent 2017 data
data.tail()

,name,type,variable_description,variable,value,annotation,year,date,geography_id,change_rate
3152,"Scenic CDP, Arizona",place,Median Household Income,B19013_001E,37174.0,NaN,2017,2017-12-31,1600000US0464650,-1.471310e+01
3153,"Hackberry CDP, Arizona",place,Median Household Income,B19013_001E,46985.0,NaN,2017,2017-12-31,1600000US0430830,3.216098e+00
3154,"Katherine CDP, Arizona",place,Median Household Income,B19013_001E,32857.0,NaN,2017,2017-12-31,1600000US0436920,-8.090566e-01
3155,"Wikieup CDP, Arizona",place,Median Household Income,B19013_001E,-666666666.0,Either no sample observations or too few sampl...,2017,2017-12-31,1600000US0482880,-1.741608e+06
3156,"Lazy Y U CDP, Arizona",place,Median Household Income,B19013_001E,126532.0,NaN,2017,2017-12-31,1600000US0440400,2.739570e+00


## Publish dataset directly from the file
- Provide a name for dataset (minimum requirement)
- Set description as `source = file`

In [21]:
with open(file_name, 'rb') as file:
# Upload + Transform step

    # revision is the *change* to the view in the catalog, which has not yet been applied.
    # output is the OutputSchema, which is a change to data which can be applied via the revision
    (revision, output) = Socrata(auth).create(
        name = dataset_name,
        description = dataset_description
    ).csv(file)

# Apply the revision - this will make it public and available to make
# visualizations from
(ok, job) = revision.apply(output_schema = output)

# Now we can get the unique dataset id
dataset_id = revision.view_id()
dataset_url = 'https://' + domain + '/d/' + dataset_id

print(dataset_url)

https://macondointernal.demo.socrata.com/d/cfvy-xet3


## Publish dataset directly from the Pandas dataframe
- Keep the same dataset name
- Set description as `source = pandas`
- Set any infinite values that may exist in `change_rate` to NA
- Set NaNs to blanks

In [12]:
pd.options.mode.use_inf_as_na = True
data = data.fillna('')

dataset_description = 'source = pandas'

(revision, output) = Socrata(auth).create(
    name = dataset_name,
    description = dataset_description
).df(data)

# Apply the revision - this will make it public and available to make
# visualizations from
(ok, job) = revision.apply(output_schema = output)

# Now we can get the unique dataset id
dataset_id = revision.view_id()
dataset_url = 'https://' + domain + '/d/' + dataset_id

print(dataset_url)

https://macondointernal.demo.socrata.com/d/nump-7qcy


## Make dataset private
Create a revision, https://github.com/socrata/socrata-py#revisions and set the dataset to private

In [24]:
# make it private
permission = 'private'

(ok, view) = socrata.views.lookup(dataset_id)
assert ok, view

(ok, revision) = view.revisions.create_update_revision(metadata = {
        'name': dataset_name,
        'description': dataset_description
    },permission = permission)

assert ok, revision
revision.apply()

(True, Job({'created_at': '2019-04-08T00:37:36.471155Z',
  'created_by': {'display_name': 'Student 1',
                 'email': '1@socrata.edu',
                 'user_id': '2wgn-e7yp'},
  'finished_at': None,
  'id': 365657,
  'is_edit': True,
  'job_uuid': 'edacd434-f69f-4300-98f5-7eef875c4403',
  'log': [],
  'output_schema_id': None,
  'request_id': '4e316b75e78ddf5673e8a01b017ffddb',
  'status': 'initializing',
  'updated_at': '2019-04-08T00:37:36.471166Z'}))

## Add a Transformation
Let's make Geography Type `Title Case` rather than loser case, https://github.com/socrata/socrata-py#transforming-your-data) for our dataset

In [25]:
# look up dataset
(ok, view) = socrata.views.lookup(dataset_id)
assert ok, view

(ok, revision) = view.revisions.create_replace_revision()
assert ok, revision

(ok, source) = revision.source_from_dataset()
assert ok, source

output_schema = source.get_latest_input_schema().get_latest_output_schema()

# use 
(ok, new_output_schema) = output_schema\
    .change_column_transform('type').to('title_case(`type`)').run()

revision.apply(output_schema = new_output_schema)

(True, Job({'created_at': '2019-04-08T00:37:48.157281Z',
  'created_by': {'display_name': 'Student 1',
                 'email': '1@socrata.edu',
                 'user_id': '2wgn-e7yp'},
  'finished_at': None,
  'id': 365658,
  'is_edit': True,
  'job_uuid': 'b846b4a9-f03c-4453-9dda-904c9e4e1684',
  'log': [],
  'output_schema_id': 579364,
  'request_id': '7d98a3fcdbdb05d4c8f1619e29123bfd',
  'status': 'initializing',
  'updated_at': '2019-04-08T00:37:48.157291Z'}))

## Add Descriptive Names and Descriptions to Columns
Make metadata revision, https://github.com/socrata/socrata-py#metadata-only-revisions

In [ ]:
(ok, view) = socrata.views.lookup(dataset_id)
assert ok, view

(ok, revision) = view.revisions.create_replace_revision()
assert ok, revision

(ok, source) = revision.source_from_dataset()
assert ok, source

output_schema = source.get_latest_input_schema().get_latest_output_schema()
(ok, new_output_schema) = output_schema\
    .change_column_metadata('name', 'display_name').to('Name')\
    .change_column_metadata('name', 'description').to('Geography name')\
    .change_column_metadata('type', 'display_name').to('Type')\
    .change_column_metadata('type', 'description').to('Census geography type')\
    .change_column_metadata('variable_description', 'display_name').to('Variable Description')\
    .change_column_metadata('variable_description', 'description').to('Census variable description')\
    .change_column_metadata('variable', 'display_name').to('Variable')\
    .change_column_metadata('variable', 'description').to('Census variable id')\
    .change_column_metadata('value', 'display_name').to('Value')\
    .change_column_metadata('value', 'description').to('Estimate value')\
    .change_column_metadata('annotation', 'display_name').to('Annotation')\
    .change_column_metadata('annotation', 'description').to('Census notes on statistial exceptions that occurred for a given year, variable and geography')\
    .change_column_metadata('year', 'display_name').to('Year')\
    .change_column_metadata('year', 'description').to('Survey Year')\
    .change_column_metadata('date', 'display_name').to('Date')\
    .change_column_metadata('date', 'description').to('Survey Estimate Year date')\
    .change_column_metadata('geography_id', 'display_name').to('Geography ID')\
    .change_column_metadata('geography_id', 'description').to('Unique Census geographic identifier')\
    .change_column_metadata('change_rate', 'display_name').to('Change Rate')\
    .change_column_metadata('change_rate', 'description').to('Rate of change from prior year')\
    .run()
assert ok, new_output_schema

revision.apply(output_schema = new_output_schema)